In [1]:
import os
import openai
from dotenv import load_dotenv
from sql_market_agent.agent.agent import create_sql_market_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain.callbacks import StdOutCallbackHandler
from langchain.agents import AgentType

import matplotlib.pyplot as plt
import numpy as np

load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
DB_HOST = os.getenv("POSTGRES_HOST", "localhost")
DB_PORT = os.getenv("POSTGRES_PORT", "5432")
DB_NAME = os.environ.get("POSTGRES_DB")
DB_USER = os.environ.get("POSTGRES_USER")
DB_PASSWORD = os.environ.get("POSTGRES_PASSWORD")

In [2]:
%matplotlib inline


In [4]:
# For using open ai models
openai.api_key = OPENAI_API_KEY
llm = ChatOpenAI(temperature=0, 
                 model="gpt-4-0125-preview", 
                 streaming=True, 
                 model_kwargs={"seed": 123})
sql_llm = ChatOpenAI(temperature=0, 
                 model="gpt-4-0125-preview", 
                 streaming=True, 
                 model_kwargs={"seed": 123})
code_llm = ChatOpenAI(temperature=0, 
                 model="gpt-4-0125-preview", 
                 streaming=True, 
                 model_kwargs={"seed": 123})


agent_executor = create_sql_market_agent(
    llm=llm,
    sql_llm=sql_llm,
    code_llm=code_llm,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    preinitialize_database=True,
    # to load specified stocks instead of internal default list from stocks.json
    # stocks=["CAT", "NKE", "XOM"], 
)

2024-03-02 09:35:49,351 - WARNING - Invalid format for stocks. Expected a list of strings or a list of dictionaries with 'ticker' and 'sector' keys. Will use default stocks from internal stocks.json.
2024-03-02 09:35:49,352 - INFO - DB connection string not provided, using local db on disc...
2024-03-02 09:35:49,371 - INFO - Initializing database with StockData table...
2024-03-02 09:35:49,552 - INFO - Database (sqlite) initialized.
2024-03-02 09:35:49,553 - INFO - Fetching data for analysis...


Directory './charts' already exists


2024-03-02 09:35:49,553 - INFO - No stocks provided. Will use default stocks from internal stocks.json.: [{'ticker': 'XOM', 'sector': 'Oil'}, {'ticker': 'CVX', 'sector': 'Oil'}]
...
2024-03-02 09:35:49,554 - INFO - No bonds provided. Will use default bonds from internal bonds.json.: [{'symbol': 'UNRATE', 'description': 'Unemployment Rate'}, {'symbol': 'CPIAUCSL', 'description': 'Consumer Price Index'}]
...
2024-03-02 09:36:01,634 - INFO - Data for stocks candles fetched and inserted successfully.
2024-03-02 09:36:01,642 - INFO - Processing XOM with CIK 0000034088
2024-03-02 09:36:02,547 - INFO - Processing CVX with CIK 0000093410
2024-03-02 09:36:03,688 - INFO - Processing COP with CIK 0001163165
2024-03-02 09:36:04,421 - INFO - Processing SLB with CIK 0000087347
2024-03-02 09:36:05,121 - INFO - Processing APA with CIK 0001841666
2024-03-02 09:36:05,450 - WARNING - Missing required data categories for Revenues
2024-03-02 09:36:05,450 - INFO - Skipping APA due to missing revenue data.
2

In [5]:
input = """Plot xom close prices for last 3 days"""
response = agent_executor(
            {"input": input, "chat_history": []},
            # callbacks=[StdOutCallbackHandler()],
)



/home/oleh/arkad/arkad-demo/arkad-sql-demo/notebooks/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


2024-03-02 09:36:41,888 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `sql_database_tool` with `What are the close prices for XOM for the last 3 days?`




> Entering new SQL Agent Executor chain...


2024-03-02 09:36:43,423 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

The provided Python code aims to plot the close prices of XOM for the last 3 days, saving the plot in an HTML format without displaying it, which aligns with the plotting rules for a task of type "plot". However, to fully evaluate the code against the provided rules, let's break down the requirements and assess the code accordingly:

1. **Plotting Rules Compliance**:
   - The code saves a plot in HTML format (`fig.write_html('./charts/xom_close_prices_last_3_days.html')`), which complies with the first rule.
   - The code does not include any command to show the plot (e.g., `fig.show()` is absent), which complies with the second rule.
   - The user input does not request a comparative chart, so the third rule regarding the use of different colors for different categories does not apply.
   - The data appears to be correctly displayed on the chart, with dates on the x-axis and close prices on the y-axis, which should make it easily readable for the user, complying with the fourth rule.


2024-03-02 09:37:13,429 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-02 09:37:21,168 - WARNING - Python REPL can execute arbitrary code. Use with caution.



Invoking: `PythonREPLTool` with `{'code': 'import plotly.graph_objects as go\n\ndates = ["2024-02-28", "2024-02-29", "2024-03-01"]\nclose_prices = [104.32, 104.52, 105.84]\n\nfig = go.Figure(data=go.Scatter(x=dates, y=close_prices, mode=\'lines+markers\'))\nfig.update_layout(title=\'XOM Close Prices for the Last 3 Days\', xaxis_title=\'Date\', yaxis_title=\'Close Price ($)\')\nfig.write_html(\'./charts/xom_close_prices_last_3_days.html\')', 'task_type': 'plot', 'chart_path': './charts/xom_close_prices_last_3_days.html'}`


{"output": "Succesfully executed:\n```python\nimport plotly.graph_objects as go\n\ndates = [\"2024-02-28\", \"2024-02-29\", \"2024-03-01\"]\nclose_prices = [104.32, 104.52, 105.84]\n\nfig = go.Figure(data=go.Scatter(x=dates, y=close_prices, mode='lines+markers'))\nfig.update_layout(title='XOM Close Prices for the Last 3 Days', xaxis_title='Date', yaxis_title='Close Price ($)')\nfig.write_html('./charts/xom_close_prices_last_3_days.html')```\n", "stdout": "", "stderr

2024-03-02 09:37:22,442 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The plot of XOM close prices for the last 3 days has been successfully created. You can view the plot by accessing the following link: [XOM Close Prices for the Last 3 Days](sandbox:/charts/xom_close_prices_last_3_days.html).

> Finished chain.
